In [1]:
import pandas as pd
import numpy as np

In [ ]:
#df = pd.read_csv('simvision_pixel_TDC.csv', skiprows=1) ## Skip first row SimTime = 0
df = pd.read_csv('Nov10_2022/simvision_in_pixel_TDC.csv', skiprows=1)

In [ ]:
df

In [ ]:
### Original column names
### 'SimTime',
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_CAL[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_TOA[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol_inst_1.Pixel_inst_2.PixelAnalog_inst.TDC_TOT[8:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_CAL[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_TOA[9:0]'
### 'ETROC2_tb.ETROC2_inst.PixelMatrix_HTree_inst.PixelCol1_inst_14.Pixel1_inst_14.PixelAnalog1_inst.TDC_TOT[8:0]'

### Replace column names (only when TDC file has been opened)
df.columns = ['SimTime', 'Row1Col2_TDC_CAL', 'Row1Col2_TDC_TOA', 'Row1Col2_TDC_TOT', 
              'Row14Col14_TDC_CAL', 'Row14Col14_TDC_TOA', 'Row14Col14_TDC_TOT']

In [ ]:
df.head()

In [ ]:
### Covert hex to decimal
row1col2_cal = df['Row1Col2_TDC_CAL'].squeeze()
row1col2_toa = df['Row1Col2_TDC_TOA'].squeeze()
b16 = lambda x: int(x, 16)
row1col2_cal = row1col2_cal.apply(b16)
row1col2_toa = row1col2_toa.apply(b16)

In [ ]:
### plotting
import matplotlib.pyplot as plt
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("1st row, 2nd column pixel TDC values", fontsize=14)
ax1.hist(row1col2_cal)
ax1.set_xlabel('CAL code', fontsize=12)

ax2.scatter(row1col2_toa, df['Row1Col2_TDC_TOT'])
ax2.set_xlim(425, 438)
ax2.grid()
ax2.set_xlabel('TOA code', fontsize=12)
ax2.set_ylabel('TOT code', fontsize=12)

In [ ]:
### Covert hex to decimal
row14col14_cal = df['Row14Col14_TDC_CAL'].squeeze()
row14col14_toa = df['Row14Col14_TDC_TOA'].squeeze()
row14col14_cal = row14col14_cal.apply(b16)
row14col14_toa = row14col14_toa.apply(b16)

In [ ]:
### plotting
import matplotlib.pyplot as plt
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle("14th row, 14th column pixel TDC values", fontsize=14)
ax1.hist(row14col14_cal)
ax1.set_xlabel('CAL code', fontsize=12)

ax2.scatter(row14col14_toa, df['Row14Col14_TDC_TOT'])
ax2.grid()
ax2.set_xlim(425, 438)
ax2.set_xlabel('TOA code', fontsize=12)
ax2.set_ylabel('TOT code', fontsize=12)

In [ ]:
df2 = pd.read_csv('simvision_fastcmd_data.csv', skiprows=1)

In [ ]:
### Original column names
### 'SimTime' 
### 'ETROC2_tb.FCGenLocal_inst.FastComByte[7:0]'

### Replace column names (only when fast command file has been opened)
df2.columns = ['SimTime', 'FastComByte']
df2.head()

In [ ]:
df2['FastComByte']
### F0 (1111 0000): Idle
### 69 (0110 1001): Charge injection
### 5A (0101 1010): BCR (Bunch counter reset at BC0, reset the counter to zero)
### 96 (1001 0110): L1A
### Reference: https://indico.cern.ch/event/1127562/contributions/4904781/attachments/2454504/4319592/ETROCemulator%20slides20220921.pdf

In [ ]:
### create a list of our conditions
conditions = [
    (df2['FastComByte'] == 'F0'),
    (df2['FastComByte'] == '69'),
    (df2['FastComByte'] == '5A'),
    (df2['FastComByte'] == '96')
    ]

### create a list of the values we want to assign for each condition
values = ['Idle', 'Charge injection', 'BCR', 'L1A']

### create a new column and use np.select to assign values to it using our lists as arguments
df2['fcmd_defn'] = np.select(conditions, values)

### display updated DataFrame
print(df2)

In [2]:
#df3 = pd.read_csv('simvision.csv', skiprows=1)
df3 = pd.read_csv('Nov10_2022/simvision_output_TDC.csv', skiprows=1)
df3.columns = ['SimTime', 'DOR', 'DOL']
df3 = df3.drop(columns=['DOL'])
df3 = df3.iloc[:-1 , :] ## drop the last row

In [3]:
sim = df3.diff(periods=-1).fillna(0)
sim['SimTime'] = sim['SimTime'].abs()*0.001
sim['nth'] = sim['SimTime']/1560.
sim.head(10)

,SimTime,DOR,nth
0,1560.0,1.0,1.0
1,1560.0,-1.0,1.0
2,3120.0,1.0,2.0
3,23400.0,-1.0,15.0
4,1560.0,1.0,1.0
5,3120.0,-1.0,2.0
6,4680.0,1.0,3.0
7,1560.0,-1.0,1.0
8,1560.0,1.0,1.0
9,4680.0,-1.0,3.0


In [4]:
df3['diff'] = sim['SimTime'] ## Time difference between (nth, (n+1)th) row, n = 0, 1, 2, ...
df3['cycle'] = sim['nth']
df3.head(10)

,SimTime,DOR,diff,cycle
0,133809921000,1,1560.0,1.0
1,133811481000,0,1560.0,1.0
2,133813041000,1,3120.0,2.0
3,133816161000,0,23400.0,15.0
4,133839561000,1,1560.0,1.0
5,133841121000,0,3120.0,2.0
6,133844241000,1,4680.0,3.0
7,133848921000,0,1560.0,1.0
8,133850481000,1,1560.0,1.0
9,133852041000,0,4680.0,3.0


In [5]:
### Repeat rows in a pandas DataFrame based on column value
### https://stackoverflow.com/questions/47336704/repeat-rows-in-a-pandas-dataframe-based-on-column-value
bit_df = df3.reindex(df3.index.repeat(df3.cycle))
bit_df = bit_df.drop(columns=['SimTime', 'diff', 'cycle'])
#bit_df = bit_df.iloc[20:, :] ## drop the first 20 rows
bit_df = bit_df.reset_index(drop=True)
bit_df.head(25)

,DOR
0,1
1,0
2,1
3,1
4,0
5,0
6,0
7,0
8,0
9,0


In [6]:
### 3C5C pattern recognition
### Only check whether the data has 3C5C pattern at anywhere or not
### 3 C 5 C (hex)
### 0011 1100 0101 1100 (binary)
pattern = [0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0] ## Since raw data bit is reversed!

matched = bit_df.rolling(len(pattern)).apply(lambda x: all(np.equal(x, pattern)), raw=True)
matched = matched.sum(axis = 1).astype(bool)   #Sum to perform boolean OR
idx_matched = np.where(matched)[0]
subset = [(match-len(pattern)+1, match+1, match+25) for match in idx_matched]
#subset = [range(match-len(pattern)+1, match+1) for match in idx_matched]   
#aligned_data = pd.concat([aligned_data.iloc[subs,:] for subs in subset], axis = 0)
#aligned_data.reset_index(inplace=True, drop=True)

In [7]:
subset

[(20, 36, 60),
 (60, 76, 100),
 (100, 116, 140),
 (140, 156, 180),
 (180, 196, 220),
 (220, 236, 260),
 (260, 276, 300),
 (300, 316, 340),
 (340, 356, 380),
 (380, 396, 420),
 (420, 436, 460),
 (460, 476, 500),
 (500, 516, 540),
 (540, 556, 580),
 (580, 596, 620),
 (620, 636, 660),
 (660, 676, 700),
 (700, 716, 740),
 (740, 756, 780),
 (780, 796, 820),
 (820, 836, 860),
 (860, 876, 900),
 (900, 916, 940),
 (940, 956, 980),
 (980, 996, 1020),
 (1020, 1036, 1060),
 (1060, 1076, 1100),
 (1100, 1116, 1140),
 (1140, 1156, 1180),
 (1180, 1196, 1220),
 (1220, 1236, 1260),
 (1260, 1276, 1300),
 (1300, 1316, 1340),
 (1340, 1356, 1380),
 (1380, 1396, 1420),
 (1420, 1436, 1460),
 (1460, 1476, 1500),
 (1500, 1516, 1540),
 (1540, 1556, 1580),
 (1580, 1596, 1620),
 (1620, 1636, 1660),
 (1660, 1676, 1700),
 (1700, 1716, 1740),
 (1740, 1756, 1780),
 (1780, 1796, 1820),
 (1820, 1836, 1860),
 (1860, 1876, 1900),
 (1900, 1916, 1940),
 (1940, 1956, 1980),
 (1980, 1996, 2020),
 (2020, 2036, 2060),
 (2060, 

In [8]:
output = []
tmp = 0

for subs in subset:
    bit16 = bit_df.iloc[range(subs[0], subs[1]), :].values.flatten().tolist()
    bit24 = bit_df.iloc[range(subs[1], subs[2]), :].values.flatten().tolist()
    bit16.reverse() ## Raw data bit order is reversed!
    bit24.reverse() ## Raw data bit order is reversed!
    s1 = ''.join([str(x) for x in bit16])
    s2 = ''.join([str(x) for x in bit24])
    output.append([s1, s2, format(int(s1, 2), 'X'), format(int(s2, 2), 'X')])

In [9]:
import os
from datetime import date

today = str(date.today())
os.system('mkdir -p output/')

new_df = pd.DataFrame(columns=['bit16', 'bit24', 'bit16_hex', 'bit24_hex'], data=output)
new_df.to_csv('output/result_'+today+'.csv', index=False)

In [10]:
## Read data
res_df = pd.read_csv('output/result.csv')
res_df.head(10)

,bit16,bit24,bit16_hex,bit24_hex
0,11110001011100,100000000000000011010001,3C5C,8000D1
1,11110001011100,100000000000000011010110,3C5C,8000D6
2,11110001011100,100000000000000011011001,3C5C,8000D9
3,11110001011100,100000000000000011011011,3C5C,8000DB
4,11110001011100,100000000000000011011110,3C5C,8000DE
5,11110001011100,100000000000000011100000,3C5C,8000E0
6,11110001011100,100000000000000011100011,3C5C,8000E3
7,11110001011100,100000000000000011100101,3C5C,8000E5
8,11110001011100,100000000000000011101000,3C5C,8000E8
9,11110001011100,100000000000000011101010,3C5C,8000EA


In [11]:
res_df.tail(10)

,bit16,bit24,bit16_hex,bit24_hex
53989,11110001011100,100010110100100111011101,3C5C,8B49DD
53990,11110001011100,100010110100100111100000,3C5C,8B49E0
53991,11110001011100,100010110100100111100010,3C5C,8B49E2
53992,11110001011100,100010110100100111100101,3C5C,8B49E5
53993,11110001011100,100010110100100111100111,3C5C,8B49E7
53994,11110001011100,100010110100100111101010,3C5C,8B49EA
53995,11110001011100,100010110100100111101100,3C5C,8B49EC
53996,11110001011100,100010110100100111101111,3C5C,8B49EF
53997,11110001011100,100010110100100111110001,3C5C,8B49F1
53998,11110001011100,100010110100100111110100,3C5C,8B49F4


In [19]:
res1 = pd.read_csv('Nov10_2022/myRes.txt', names=['datawords_Jongho'])
res2 = pd.read_csv('Nov10_2022/dataward_neha.txt', names=['datawords_Neha'])
res = pd.concat([res1, res2], axis=1)
res['same'] = np.where(res['datawords_Jongho'] == res['datawords_Neha'], True, False)
res.head()
res.to_csv('output/compare_result.csv', index=False)

In [12]:
### 6AF3 pattern
### 6 A F 3
### 0110 1010 1111 0011
pattern = [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0] ## Since raw data bit is reversed!

matched = bit_df.rolling(len(pattern)).apply(lambda x: all(np.equal(x, pattern)), raw=True)
matched = matched.sum(axis = 1).astype(bool)   #Sum to perform boolean OR
idx_matched = np.where(matched)[0]
subset_trailer = [(match-len(pattern)+1, match+1, match+25) for match in idx_matched]

In [13]:
subset_trailer

[(209672, 209688, 209712),
 (323712, 323728, 323752),
 (494792, 494808, 494832),
 (608832, 608848, 608872),
 (779912, 779928, 779952),
 (893952, 893968, 893992),
 (1065032, 1065048, 1065072),
 (1179072, 1179088, 1179112),
 (1350147, 1350163, 1350187),
 (1464187, 1464203, 1464227),
 (1635267, 1635283, 1635307),
 (1749307, 1749323, 1749347),
 (1920387, 1920403, 1920427),
 (2034427, 2034443, 2034467)]

In [ ]:
#### Backup code

#subset1 = []
#subset2 = []

#output = []
#for i in range(int(bit_df.shape[0]/40)):
#    x1, x2 = 0+40*i, 16+40*i
#    y1, y2 = 16+40*i, 40+40*i
#    bit16 = bit_df.iloc[range(x1, x2), :].values.flatten().tolist()
#    bit24 = bit_df.iloc[range(y1, y2), :].values.flatten().tolist()
#    bit16.reverse() ## Raw data bit order is reversed!
#    bit24.reverse() ## Raw data bit order is reversed!
#    s1 = ''.join([str(x) for x in bit16])
#    s2 = ''.join([str(x) for x in bit24])
#    output.append([s1, s2, format(int(s1, 2), 'X'), format(int(s2, 2), 'X')])
    
#    if i < 10:
#        print(s1, s2, format(int(s1, 2), 'X'), format(int(s2, 2), 'X'))

#new_df = pd.DataFrame(columns=['bit16', 'bit24', 'bit16_hex', 'bit24_hex'], data=output)
#new_df.to_csv('result1.csv', index=False)